In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
age_gender=pd.read_csv('age_gender_info.csv')

In [ ]:
print("train shape: {}, test shape: {}, submission shape: {}, age_gender_info shape: {}".format(train.shape,test.shape, submission.shape, age_gender.shape))

train shape: (2952, 15), test shape: (1022, 14), submission shape: (150, 2), age_gender_info shape: (16, 23)


In [ ]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [ ]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [ ]:
age_gender.head()

,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,경상북도,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,0.072613,0.087149,0.072146,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,경상남도,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,대전광역시,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,0.067925,0.085751,0.068819,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066
3,경기도,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,0.078355,0.077345,0.064859,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,전라북도,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,0.066645,0.081408,0.074382,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013


In [ ]:
row=age_gender.loc[0,:]
print(row)

지역                  경상북도
10대미만(여자)      0.0301581
10대미만(남자)      0.0331946
10대(여자)        0.0563456
10대(남자)        0.0613605
20대(여자)        0.0600965
20대(남자)        0.0678593
30대(여자)        0.0534328
30대(남자)         0.049572
40대(여자)        0.0836596
40대(남자)        0.0726131
50대(여자)        0.0871495
50대(남자)         0.072146
60대(여자)        0.0826841
60대(남자)        0.0638885
70대(여자)        0.0477172
70대(남자)        0.0301719
80대(여자)        0.0293613
80대(남자)        0.0112114
90대(여자)       0.00557823
90대(남자)       0.00155256
100대(여자)     0.000233571
100대(남자)     1.37395e-05
Name: 0, dtype: object


In [ ]:
total=0
for i in range(1,len(row)):
    total=total+row[i]

In [ ]:
total

1.0000000000000002

In [ ]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [ ]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수', '등록차량수'],
      dtype='object')

In [ ]:
train_cols=['임대건물구분', '지역', '공급유형', 
       '자격유형', '지하철',
       '버스']

for col in train_cols:
    print("{} 의 unique 값 ::: {}".format(col,pd.unique(train[col])))

임대건물구분 의 unique 값 ::: ['아파트' '상가']
지역 의 unique 값 ::: ['경상북도' '경상남도' '대전광역시' '경기도' '전라북도' '강원도' '광주광역시' '충청남도' '부산광역시' '제주특별자치도'
 '울산광역시' '충청북도' '전라남도' '대구광역시' '서울특별시' '세종특별자치시']
공급유형 의 unique 값 ::: ['국민임대' '공공임대(50년)' '영구임대' '임대상가' '공공임대(10년)' '공공임대(분납)' '장기전세' '공공분양'
 '행복주택' '공공임대(5년)']
자격유형 의 unique 값 ::: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
지하철 의 unique 값 ::: [ 0.  1. nan  2.  3.]
버스 의 unique 값 ::: [ 3.  1.  2.  6. 10.  5.  4.  7. 12. 14.  8.  0. 20. 11. 16. 15. nan 19.]


In [ ]:
train.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금         0
임대료           0
지하철         211
버스            0
단지내주차면수       0
등록차량수         0
dtype: int64

In [ ]:
test.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64

In [ ]:
le = LabelEncoder()
le.fit(train.loc[:,train.columns=='지역'])
train['지역']=le.transform(train.loc[:,train.columns=='지역'])
test['지역']=le.transform(test.loc[:,test.columns=='지역'])

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
train.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,아파트,3,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,3,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,3,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [ ]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

In [ ]:
print(train['전용면적'].unique())
print("==========================")
print(test['전용면적'].unique())

[ 35.  50.  55.  30.  45.  40.  25.  70.  10.  15.  20. 315.  60. 135.
  75. 105. 580. 125. 240. 400. 245. 405.  80.  65.]
[ 35.  45.  50.  30.  55.  25.  75. 580.  15.  20.  40. 400.  60.   5.
  10. 245. 250.  80. 240.  70.]


#Baseline(random forest)

In [ ]:
columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [ ]:
area_columns

['면적_35.0',
 '면적_50.0',
 '면적_55.0',
 '면적_30.0',
 '면적_45.0',
 '면적_40.0',
 '면적_25.0',
 '면적_70.0',
 '면적_10.0',
 '면적_15.0',
 '면적_20.0',
 '면적_315.0',
 '면적_60.0',
 '면적_135.0',
 '면적_75.0',
 '면적_105.0',
 '면적_580.0',
 '면적_125.0',
 '면적_240.0',
 '면적_400.0',
 '면적_245.0',
 '면적_405.0',
 '면적_80.0',
 '면적_65.0']

In [ ]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [ ]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

423it [00:12, 34.60it/s]
150it [00:04, 35.85it/s]


In [ ]:
new_train

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_35.0,면적_50.0,면적_55.0,면적_30.0,면적_45.0,면적_40.0,면적_25.0,면적_70.0,면적_10.0,면적_15.0,면적_20.0,면적_315.0,면적_60.0,면적_135.0,면적_75.0,면적_105.0,면적_580.0,면적_125.0,면적_240.0,면적_400.0,면적_245.0,면적_405.0,면적_80.0,면적_65.0,등록차량수
0,C2483,900.0,38.0,3.0,1425.0,0.0,3.0,149.0,665.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.0
1,C2515,545.0,17.0,2.0,624.0,0.0,3.0,80.0,132.0,0.0,276.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0
2,C1407,1216.0,13.0,6.0,1285.0,1.0,1.0,0.0,124.0,0.0,390.0,340.0,362.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1064.0
3,C1945,755.0,6.0,1.0,734.0,1.0,3.0,240.0,303.0,212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730.0
4,C1470,696.0,14.0,12.0,645.0,0.0,2.0,254.0,246.0,0.0,0.0,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,C2586,90.0,7.0,13.0,66.0,0.0,3.0,36.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0
419,C2035,492.0,24.0,0.0,521.0,0.0,1.0,156.0,0.0,0.0,0.0,180.0,0.0,156.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,246.0
420,C2020,40.0,7.0,7.0,25.0,1.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
421,C2437,90.0,12.0,15.0,30.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0


In [ ]:
new_train.isnull().sum()

단지코드         0
총세대수         0
공가수          0
지역           0
단지내주차면수      0
지하철         20
버스           1
면적_35.0      0
면적_50.0      0
면적_55.0      0
면적_30.0      0
면적_45.0      0
면적_40.0      0
면적_25.0      0
면적_70.0      0
면적_10.0      0
면적_15.0      0
면적_20.0      0
면적_315.0     0
면적_60.0      0
면적_135.0     0
면적_75.0      0
면적_105.0     0
면적_580.0     0
면적_125.0     0
면적_240.0     0
면적_400.0     0
면적_245.0     0
면적_405.0     0
면적_80.0      0
면적_65.0      0
등록차량수        0
dtype: int64

In [ ]:
new_train = new_train.fillna(-1)
new_test = new_test.fillna(-1)

In [ ]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [ ]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
pred = model.predict(x_test)
submission['num'] = pred
submission.to_csv('baseline.csv', index=False)

# 방법 1

In [42]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
age_gender=pd.read_csv('age_gender_info.csv')

In [43]:
print("train shape: {}, test shape: {}, submission shape: {}, age_gender_info shape: {}".format(train.shape,test.shape, submission.shape, age_gender.shape))

train shape: (2952, 15), test shape: (1022, 14), submission shape: (150, 2), age_gender_info shape: (16, 23)


In [44]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [45]:
train.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       569
임대료         569
지하철         211
버스            4
단지내주차면수       0
등록차량수         0
dtype: int64

In [46]:
test.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64

In [47]:
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


##임대보증금, 임대료

In [48]:
train['임대보증금']=train['임대보증금'].fillna(-1)
train['임대료']=train['임대료'].fillna(-1)

test['임대보증금']=test['임대보증금'].fillna(-1)
test['임대료']=test['임대료'].fillna(-1)

In [69]:
for col in train.columns:
    print(col)
    # train[train[col].str.contains('-')] 
    idx = train[train[col] == '-'].index
    print(idx)

단지코드
Int64Index([], dtype='int64')
총세대수
Int64Index([], dtype='int64')
임대건물구분
Int64Index([], dtype='int64')
지역
Int64Index([], dtype='int64')
공급유형
Int64Index([], dtype='int64')
전용면적
Int64Index([], dtype='int64')
전용면적별세대수
Int64Index([], dtype='int64')
공가수
Int64Index([], dtype='int64')
자격유형
Int64Index([], dtype='int64')
임대보증금
Int64Index([2547, 2548, 2549, 2550, 2551, 2680, 2681, 2682, 2683, 2906, 2907,
            2908],
           dtype='int64')
임대료
Int64Index([2043, 2044, 2045, 2046, 2047, 2048, 2100, 2101, 2102, 2547, 2548,
            2549, 2550, 2551, 2680, 2681, 2682, 2683, 2906, 2907, 2908],
           dtype='int64')
지하철
Int64Index([], dtype='int64')
버스
Int64Index([], dtype='int64')
단지내주차면수
Int64Index([], dtype='int64')
등록차량수
Int64Index([], dtype='int64')


In [70]:
for col in test.columns:
    print(col)
    # train[train[col].str.contains('-')] 
    idx = test[test[col] == '-'].index
    print(idx)

단지코드
Int64Index([], dtype='int64')
총세대수
Int64Index([], dtype='int64')
임대건물구분
Int64Index([], dtype='int64')
지역
Int64Index([], dtype='int64')
공급유형
Int64Index([], dtype='int64')
전용면적
Int64Index([], dtype='int64')
전용면적별세대수
Int64Index([], dtype='int64')
공가수
Int64Index([], dtype='int64')
자격유형
Int64Index([], dtype='int64')
임대보증금
Int64Index([1005, 1006, 1014, 1015, 1016, 1017], dtype='int64')
임대료
Int64Index([1005, 1006, 1014, 1015, 1016, 1017], dtype='int64')
지하철
Int64Index([], dtype='int64')
버스
Int64Index([], dtype='int64')
단지내주차면수
Int64Index([], dtype='int64')


In [71]:
train[train['임대보증금'] == '-']=-1
train[train['임대료'] == '-']=-1
test[test['임대보증금'] == '-']=-1
test[test['임대료'] == '-']=-1

## 버스

In [49]:
train['버스']=train['버스'].fillna(-1)

## 지하철

In [50]:
print('지하철 결측치 단지 지역: ', train.loc[train['지하철'].isnull()]['지역'].unique())
print('지하철 결측치 단지 지역: ', test.loc[test['지하철'].isnull()]['지역'].unique())

지하철 결측치 단지 지역:  ['충청남도' '대전광역시' '경상남도']
지하철 결측치 단지 지역:  ['충청남도' '대전광역시']


In [51]:
# for i in range(len(train)):
#     if train['지하철'][i] == np.nan :
#         if train['지역']=='충청남도':
#             train['지하철'][i]=-1
#         if train['지역']=='대전광역시':
#             train['지하철'][i]=-2
#         if train['지역']=='경상남도':
#             train['지하철'][i]=-3

In [52]:
train['지하철'][(train['지하철'].isnull())&(train['지역']=='충청남도')] =-1
train['지하철'][(train['지하철'].isnull())&(train['지역']=='대전광역시')] = -2
train['지하철'][(train['지하철'].isnull())&(train['지역']=='경상남도')] = -3

test['지하철'][(test['지하철'].isnull())&(test['지역']=='충청남도')] = -1
test['지하철'][(test['지하철'].isnull())&(test['지역']=='대전광역시')] = -2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

## 전용면적

In [53]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

In [54]:
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,35.0,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,35.0,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,50.0,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,50.0,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,50.0,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,45.0,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,50.0,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,50.0,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,50.0,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


In [ ]:
# for i, code in tqdm(enumerate(train['단지코드'].unique())):
#     temp = train[train['단지코드']==code]
#     temp.index = range(temp.shape[0])
#     for col in columns:
#         new_train.loc[i, col] = temp.loc[0, col]
    
#     for col in area_columns:
#         area = float(col.split('_')[-1])
#         new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

##임대건물구분, 지역, 공급유형

In [55]:
train_cols=['임대건물구분', '지역', '공급유형']

for col in train_cols:
    print("{} 의 unique 값 ::: {}".format(col,pd.unique(train[col])))

임대건물구분 의 unique 값 ::: ['아파트' '상가']
지역 의 unique 값 ::: ['경상북도' '경상남도' '대전광역시' '경기도' '전라북도' '강원도' '광주광역시' '충청남도' '부산광역시' '제주특별자치도'
 '울산광역시' '충청북도' '전라남도' '대구광역시' '서울특별시' '세종특별자치시']
공급유형 의 unique 값 ::: ['국민임대' '공공임대(50년)' '영구임대' '임대상가' '공공임대(10년)' '공공임대(분납)' '장기전세' '공공분양'
 '행복주택' '공공임대(5년)']


In [56]:
le = LabelEncoder()
le.fit(train.loc[:,train.columns=='공급유형'].values)
train['공급유형']=le.transform(train.loc[:,train.columns=='공급유형'].values)
test['공급유형']=le.transform(test.loc[:,test.columns=='공급유형'].values)

le.fit(train.loc[:,train.columns=='지역'])
train['지역']=le.transform(train.loc[:,train.columns=='지역'])
test['지역']=le.transform(test.loc[:,test.columns=='지역'])

le.fit(train.loc[:,train.columns=='임대건물구분'].values)
train['임대건물구분']=le.transform(train.loc[:,train.columns=='임대건물구분'].values)
test['임대건물구분']=le.transform(test.loc[:,test.columns=='임대건물구분'].values)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [57]:
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,1,3,5,35.0,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,1,3,5,35.0,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,1,3,5,50.0,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,1,3,5,50.0,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,1,3,5,50.0,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,1,0,5,45.0,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,1,0,5,50.0,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,1,0,5,50.0,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,1,0,5,50.0,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


##자격유형

In [58]:
print(train['자격유형'].unique())
print(test['자격유형'].unique())

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['H' 'A' 'E' 'C' 'D' nan 'G' 'I' 'J' 'K' 'L' 'M' 'N']


In [59]:
qualification_type_map={'A':0, 'B':1, 'C':2 ,'D':3 ,'E':4, 'F':5 ,'G':6, 'H':7, 'I':8, 'J':9, 'K':10, 'L':11, 'M':12, 'N':13, 'O':14, np.nan:-1}
train['자격유형']=train['자격유형'].map(qualification_type_map)
test['자격유형']=test['자격유형'].map(qualification_type_map)

In [60]:
train.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
등록차량수       0
dtype: int64

In [61]:
test.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
자격유형        0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
dtype: int64

In [62]:
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,1,3,5,35.0,134,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,1,3,5,35.0,15,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,1,3,5,50.0,385,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,1,3,5,50.0,15,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,1,3,5,50.0,41,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,1,0,5,45.0,19,7.0,0,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,1,0,5,50.0,34,7.0,0,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,1,0,5,50.0,34,7.0,0,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,1,0,5,50.0,114,7.0,0,14005000,142310,0.0,1.0,166.0,146.0


##model

In [73]:
x_train = train.iloc[:, 1:-1]
y_train = train.iloc[:,-1]
x_test = test.iloc[:,1:]

In [74]:
svr.fit(x_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [75]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
svr=SVR()
parameters={'C':[0.5,0.7,1,3,10,50,100]}
clf = GridSearchCV(svr, parameters)

In [76]:
clf.fit(x_train,y_train)
pred = clf.predict(x_test)
submission['num'] = pred
submission.to_csv('baseline.csv', index=False)

ValueError: ignored